In [ ]:
import os
import requests
import dotenv
import time
import numpy as np
import pandas as pd
import yfinance
import alpaca_trade_api as alpaca_api
import krakenex
import pykrakenapi
import pandas_montecarlo

%matplotlib inline

In [ ]:
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame({'ticker':tickers})
    for n in range(0, n):
        random_portfolios[f'random weights {n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'random weights {n+1}'] = random_portfolios[f'random weights {n+1}'] / sum(random_portfolios[f'random weights {n+1}'])
    return(random_portfolios)

In [ ]:
dotenv.load_dotenv()

In [ ]:
alpaca_api_key = os.getenv('alpaca_api_key')
alpaca_secret_key = os.getenv('alpaca_secret_key')
kraken_api_key = os.getenv('kraken_api_key')
kraken_secret_key = os.getenv('kraken_secret_key')

print(type(alpaca_api_key))
print(type(alpaca_secret_key))
print(type(kraken_api_key))
print(type(kraken_secret_key))

In [ ]:
#table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#sp_500_table = table[0]
#sp_500_list = sp_500_table['Symbol'].to_list()
#sp_500 = yfinance.download(sp_500_list, '2015-12-01')
#sp_500

In [ ]:
alpaca = alpaca_api.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

faangmula_tickers =  ['AAPL',  'ABNB', 'AMZN', 'FB', 'GOOG', 'LYFT', 'MSFT', 'NFLX', 'UBER']
start = pd.Timestamp('2015-12-01', tz='America/New_York').isoformat()

faangmula_original = alpaca.get_bars(faangmula_tickers, alpaca_api.TimeFrame.Day, start = start).df
faangmula_original

In [ ]:
faangmula = faangmula_original.drop(['open', 'high', 'low', 'volume', 'trade_count', 'vwap'], axis = 1).pivot(columns = 'symbol')
faangmula.index = faangmula.index.date
faangmula.columns = faangmula_tickers
faangmula

In [ ]:
# Market capitalization data in trillions of dollars, sourced from Trading View on April 6, 2022.

faangmula_caps_list = [2.804, 0.105855, 1.616, 0.60781, 1.81, 0.012576, 2.245, 0.163534, 0.065299]
faangmula_cap_portfolio = pd.DataFrame({'tickers':faangmula_tickers, 'market cap weights':[market_cap / sum(faangmula_caps_list) for market_cap in faangmula_caps_list]}).set_index('tickers')
faangmula_cap_portfolio

In [ ]:
faangmula_cap_daily_returns = faangmula.pct_change().fillna(0.0).dot(faangmula_cap_portfolio)
faangmula_cap_daily_returns.columns = ['faangmula index']
faangmula_cap_daily_returns

In [ ]:
random_faangmula_portfolios = random_portfolios_generator(faangmula_tickers, 1000)
random_faangmula_portfolios

In [ ]:
random_faangmula_daily_returns_list = []
for n in range(0, (len(random_faangmula_portfolios.columns) - 1)):
        random_faangmula_daily_returns_list.append(faangmula.pct_change().fillna(0.0).dot(list(random_faangmula_portfolios.loc[:, f'random weights {n + 1}'])))

random_faangmula_daily_returns = pd.concat(random_faangmula_daily_returns_list, axis = 'columns', join = 'inner')

for n in range(0, (len(random_faangmula_portfolios.columns) - 1)):
        random_faangmula_daily_returns = random_faangmula_daily_returns.rename(columns = {n: f'random faangmula portfolio {n + 1}'})

random_faangmula_daily_returns

In [ ]:
faangmula_daily_returns = pd.concat([faangmula_cap_daily_returns, random_faangmula_daily_returns], axis = 1, join = 'inner')
faangmula_daily_returns

In [ ]:
faangmula_correlations = faangmula_daily_returns.corr()
faangmula_correlations

In [ ]:
low_correlation_faangmula_original = faangmula_correlations.sort_values('faangmula index')['faangmula index'].head(10).reset_index().rename({'index': 'portfolio', 'faangmula index': 'faangmula index correlation'}, axis = 1)
low_correlation_faangmula_original.loc[len(low_correlation_faangmula_original.values)] = ['faangmula index', 1.0]
low_correlation_faangmula_original

In [ ]:
variances = []
covariances = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    variance = faangmula_daily_returns[f'{portfolio}'].var()
    covariance = faangmula_daily_returns[f'{portfolio}'].cov(faangmula_daily_returns['faangmula index'])
    variances.append(variance)
    covariances.append(covariance)

lc_intermediate_1 = pd.concat([low_correlation_faangmula_original, pd.DataFrame(variances)], axis = 1).rename({0: 'portfolio variance'}, axis = 1)
lc_intermediate_2 = pd.concat([lc_intermediate_1, pd.DataFrame(covariances)], axis = 1).rename({0: 'faangmula index covariance'}, axis = 1)
lc_intermediate_2

In [ ]:
betas = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    beta = lc_intermediate_2.set_index('portfolio').at[f'{portfolio}', 'faangmula index covariance'] / lc_intermediate_2.set_index('portfolio').at[f'{portfolio}', 'portfolio variance']
    betas.append(beta)

lc_intermediate_3 = pd.concat([lc_intermediate_2, pd.DataFrame(betas)], axis = 1).rename({0: 'faangmula index beta'}, axis = 1)
lc_intermediate_3

In [ ]:
average_returns = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    average_return = faangmula_daily_returns[f'{portfolio}'].mean()
    average_returns.append(average_return)

lc_intermediate_4 = pd.concat([lc_intermediate_3, pd.DataFrame(average_returns)], axis = 1).rename({0: 'average daily return'}, axis = 1)
lc_intermediate_4

In [ ]:
stds = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    std = faangmula_daily_returns[f'{portfolio}'].std()
    stds.append(std)

lc_intermediate_5 = pd.concat([lc_intermediate_4, pd.DataFrame(stds)], axis = 1).rename({0: 'standard deviation'}, axis = 1)
lc_intermediate_5

In [ ]:
zero_sharpes = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    zero_sharpe = (lc_intermediate_5.set_index('portfolio').at[f'{portfolio}', 'average daily return'] - 0) / lc_intermediate_5.set_index('portfolio').at[f'{portfolio}', 'standard deviation']
    zero_sharpes.append(zero_sharpe)

lc_intermediate_6 = pd.concat([lc_intermediate_5, pd.DataFrame(zero_sharpes)], axis = 1).rename({0: 'zero sharpe ratio'}, axis = 1)
lc_intermediate_6

In [ ]:
faangmula_sharpes = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    faangmula_sharpe = (lc_intermediate_6.set_index('portfolio').at[f'{portfolio}', 'average daily return'] - lc_intermediate_5.set_index('portfolio').at['faangmula index', 'average daily return']) / lc_intermediate_6.set_index('portfolio').at[f'{portfolio}', 'standard deviation']
    faangmula_sharpes.append(faangmula_sharpe)
lc_intermediate_7 = pd.concat([lc_intermediate_6, pd.DataFrame(faangmula_sharpes)], axis = 1).rename({0: 'faangmula sharpe ratio'}, axis = 1)
lc_intermediate_7

In [ ]:
cumulative_returns = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    cumulative_return = (1 + faangmula_daily_returns[f'{portfolio}']).cumprod()
    cumulative_returns.append(cumulative_return)

cumulative_returns_df = pd.DataFrame(cumulative_returns)
cumulative_returns_df

In [ ]:
lc_intermediate_7.plot(kind = 'barh', x = 'portfolio', y = 'faangmula index correlation', xlabel = '', ylabel = 'Correlation Coefficient', figsize = (20, 10), title = 'Correlations Against FAANGMULA')

In [ ]:
lc_intermediate_7.plot(kind = 'barh', x = 'portfolio', y = 'zero sharpe ratio', xlabel = '', ylabel = 'Sharpe Ratio Against Zero Return', figsize = (20, 10), title = 'Sharpe Ratios Against Zero Return')

In [ ]:
from datetime import datetime
#import questionary
import requests
import json

In [ ]:
def coinapi_data(tickers):
   
    headers = {'X-CoinAPI-Key' : 'DABB5DFC-447C-4F57-8600-147722F5C7BA'}

    coinapi_data = {}
    for ticker in tickers:
        url = f'https://rest.coinapi.io/v1/exchangerate/{ticker}/USD/history?period_id=1DAY&time_start=2019-01-01T00:00:00&time_end=2022-04-10T00:00:00&limit=900'
        response = requests.get(url, headers=headers)
        
        print(json.loads(response.content))
        ticker_data = pd.DataFrame(json.loads(response.content))

        #ticker_data['symbol'] = ticker
        ticker_data = ticker_data.rename(columns = {'time_period_end': 'timestamp', 'rate_close': 'close'})
        ticker_data = ticker_data.drop(['time_period_start', 'time_open', 'time_close', 'rate_open', 'rate_high', 'rate_low'], axis=1).reset_index()
        ticker_data = ticker_data.set_index('timestamp')
        coinapi_data[ticker] = ticker_data

        time.sleep(3)
        
    return coinapi_data

In [ ]:
tickers = ['BTC', 'ETH', 'LUNA', 'SOL', 'XRP', 'ADA', 'AVAX', 'DOT', 'DOGE', 'SHIB', 'MATIC', 'DAI', 'LTC', 'ATOM', 'LINK', 'UNI', 'TRX', 'BCH']
#two_tickers = ['BTC', 'ETH']
crypto_original = coinapi_data(tickers)

In [ ]:
crypto_original = pd.concat(crypto_original.values(), keys = crypto_original.keys(), axis = 'columns')
crypto = crypto_original.xs('close', level = 1, axis = 'columns')
crypto.index = crypto.index.astype('datetime64[ns]')
crypto

In [ ]:
crypto.to_csv('./crypto.csv')

In [ ]:
all_close = pd.concat([faangmula, crypto], axis = 1, join = 'outer')
all_close